<a href="https://colab.research.google.com/github/Farnaaaz/TenniMatchPrediction/blob/main/TennisResultPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import os
import pandas as pd
from io import StringIO
import io
import string
import random
import json
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from google.colab import drive

Filtering and Merging the TOURNAMENT data sets

In [64]:
tournaments1 = pd.read_csv('match_scores_1991-2016_unindexed_csv.csv')

In [65]:
#Choosing only data for year 2016
filtered_t1 = tournaments1[tournaments1['tourney_year_id'].str.startswith('2016-')]

In [66]:
filtered_t1.head()

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
91415,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Finals,1,1,Milos Raonic,r975,milos-raonic,...,1,64 64,2,0,12,8,0,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats
91416,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,1,Roger Federer,f324,roger-federer,...,8,61 64,2,0,12,5,0,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats
91417,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,2,Milos Raonic,r975,milos-raonic,...,7,76(5) 76(5),2,0,14,12,2,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats
91418,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,1,Roger Federer,f324,roger-federer,...,NaN,64 67(4) 64,2,1,18,15,0,1,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats
91419,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,2,Bernard Tomic,ta46,bernard-tomic,...,2,63 16 63,2,1,13,12,0,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats


In [67]:
filtered_t1_rows = filtered_t1.shape[0]

In [68]:
print(f"The filtered ATP1 DataFrame has {filtered_t1_rows} rows.")

The filtered ATP1 DataFrame has 3944 rows.


In [69]:
tournaments2 = pd.read_csv('match_scores_2017_unindexed_csv.csv')

In [70]:
t2_rows = tournaments2.shape[0]
print(f"The filtered ATP2 DataFrame has {t2_rows} rows.")

The filtered ATP2 DataFrame has 3830 rows.


In [71]:
#Joining two tables vertically
merged_tournaments = pd.concat([filtered_t1, tournaments2], ignore_index=True)

In [72]:
merged_tournaments.head()

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Finals,1,1,Milos Raonic,r975,milos-raonic,...,1,64 64,2,0,12,8,0,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats
1,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,1,Roger Federer,f324,roger-federer,...,8,61 64,2,0,12,5,0,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats
2,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,2,Milos Raonic,r975,milos-raonic,...,7,76(5) 76(5),2,0,14,12,2,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats
3,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,1,Roger Federer,f324,roger-federer,...,NaN,64 67(4) 64,2,1,18,15,0,1,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats
4,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,2,Bernard Tomic,ta46,bernard-tomic,...,2,63 16 63,2,1,13,12,0,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats


In [73]:
#verifying the merge, should be 3830 + 3944
tournament_rows = merged_tournaments.shape[0]
print(f"The filtered merged_ATP DataFrame has {tournament_rows} rows.")

The filtered merged_ATP DataFrame has 7774 rows.


Filtering and Merging the MATCH data sets

In [74]:
matches1 = pd.read_csv('match_stats_2017_unindexed_csv (1).csv')
matches2 = pd.read_csv('match_stats_1991-2016_unindexed_csv.csv')

In [75]:
#Choosing only data for year 2016
filtered_match2 = matches2[matches2['match_id'].str.startswith('2016-')]

In [76]:
filtered_match2.head()

,tourney_order,match_id,match_stats_url_suffix,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
88033,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats,01:01:00,61.0,6.0,0.0,27.0,49.0,23.0,...,10.0,22.0,1.0,1.0,8.0,9.0,14.0,49.0,41.0,104.0
88034,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats,01:27:00,87.0,6.0,6.0,34.0,60.0,28.0,...,12.0,26.0,0.0,1.0,10.0,10.0,18.0,60.0,57.0,121.0
88035,0,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats,02:08:00,128.0,7.0,1.0,52.0,83.0,45.0,...,12.0,31.0,1.0,1.0,16.0,16.0,19.0,83.0,86.0,187.0
88036,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats,01:48:00,108.0,13.0,3.0,49.0,79.0,40.0,...,11.0,30.0,0.0,4.0,12.0,12.0,20.0,79.0,77.0,163.0
88037,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats,02:00:00,120.0,14.0,1.0,62.0,97.0,45.0,...,22.0,35.0,3.0,11.0,12.0,13.0,39.0,97.0,89.0,181.0


In [77]:
match2_rows = filtered_match2.shape[0]
print(f"The filtered merged_ATP DataFrame has {match2_rows} rows.")

The filtered merged_ATP DataFrame has 3923 rows.


In [78]:
match1_rows = matches1.shape[0]
print(f"The filtered ATP2 DataFrame has {match1_rows} rows.")

The filtered ATP2 DataFrame has 3811 rows.


In [79]:
#Joining two tables vertically
merged_matches = pd.concat([filtered_match2, matches1], ignore_index=True)

In [80]:
#verifying the merge, should be 3923 + 3811
matches_rows = merged_matches.shape[0]
print(f"The filtered merged_ATP DataFrame has {matches_rows} rows.")

The filtered merged_ATP DataFrame has 7734 rows.


Merging Tournaments and Matches tables with Players table

In [81]:
players = pd.read_csv('player_overviews_unindexed_csv (1).csv')

In [82]:
# Merge for winner players
merged_df_winner = pd.merge(merged_tournaments, players, left_on='winner_player_id', right_on='player_id', how='inner')

# Merge for loser players
merged_df_loser = pd.merge(merged_tournaments, players, left_on='loser_player_id', right_on='player_id', how='inner')

# Concatenate the two merged DataFrames
final_df = pd.concat([merged_df_winner, merged_df_loser])

In [83]:
# Merge with merged_matches
final_df = pd.merge(merged_df_winner, merged_matches, on='match_id', how='inner')

# Concatenate the two merged DataFrames
final_df = pd.concat([merged_df_winner, merged_df_loser])

In [84]:
final_df.head()


,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,birth_month,birth_day,turned_pro,weight_lbs,weight_kg,height_ft,height_inches,height_cm,handedness,backhand
0,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Finals,1,1,Milos Raonic,r975,milos-raonic,...,12.0,27.0,2008.0,216.0,98.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand
1,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,2,Milos Raonic,r975,milos-raonic,...,12.0,27.0,2008.0,216.0,98.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand
2,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,4,Milos Raonic,r975,milos-raonic,...,12.0,27.0,2008.0,216.0,98.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand
3,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Round of 16,4,4,Milos Raonic,r975,milos-raonic,...,12.0,27.0,2008.0,216.0,98.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand
4,2016-580,6,australian-open,/en/scores/archive/australian-open/580/2016/re...,Quarter-Finals,3,4,Milos Raonic,r975,milos-raonic,...,12.0,27.0,2008.0,216.0,98.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand


In [86]:
selected_columns = ['tourney_year_id', 'winner_name', 'winner_player_id', 'loser_name', 'loser_player_id']
selected_df = final_df[selected_columns]

# Display the result
print(selected_df)

     tourney_year_id     winner_name winner_player_id         loser_name  \
0           2016-339    Milos Raonic             r975      Roger Federer   
1           2016-339    Milos Raonic             r975      Bernard Tomic   
2           2016-339    Milos Raonic             r975      Lucas Pouille   
3           2016-339    Milos Raonic             r975         Ivan Dodig   
4           2016-580    Milos Raonic             r975       Gael Monfils   
...              ...             ...              ...                ...   
7671       2017-6967   Nicolas Jarry             j551           Andre Sa   
7672       2017-6967     Lukas Lacko             l797   Yusuke Takahashi   
7673        2017-329   Ryan Harrison             h940   Yusuke Takahashi   
7674        2017-438  Alexey Vatutin             v717  Denis Matsukevich   
7675        2017-429     Lukas Lacko             l797     Viktor Galovic   

     loser_player_id  
0               f324  
1               ta46  
2               pf